In [ ]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RANSACRegressor, LinearRegression
import cv2
import glob as glob
import matplotlib.pyplot as plt 
from PIL import Image 
import pandas as pd 
import numpy as np 
import os 
from keras.layers.core import Dropout
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import  ResNet50
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from numpy import savetxt
from numpy import loadtxt
import random
from scipy import stats
from matplotlib.gridspec import GridSpec

In [ ]:
# Current Directory
cur_dir = "/Users/natanviana/Desktop/big_data"

# Initializing the Tranfer Learning Model Resnet50
my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights="imagenet"))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

In [ ]:
# Extract Function to save the output of images after pass inside a Resnet50 Model
def extract_vector(path,resnet_feature_list):

    im = cv2.imread(path)
    img = preprocess_input(np.expand_dims(im.copy(), axis=0))
    resnet_feature = my_new_model.predict(img)
    resnet_feature_np = np.array(resnet_feature)
    if(currentframe==0):
        resnet_feature_list = resnet_feature_np
    else:
        resnet_feature_list =np.append(resnet_feature_list,resnet_feature_np,axis=0)

    return np.array(resnet_feature_list)
    

In [ ]:
# Getting the name of the video  and the extension
nameVideo = "girosmallveryslow2.mp4"
nameVideo = nameVideo.split(".")
extension = "." + nameVideo[1]
nameVideo = nameVideo[0]

In [ ]:
# Choose the name of cluster that you want to cluster again 
cluster = "cluster2"
path_to_cluster = "/"+ nameVideo +"/clusterImages/"+ cluster

# Getting the Path to the frames inside this folder
list = os.listdir(cur_dir + path_to_cluster) # variavel
number_files = len(list)
number_files


In [ ]:
# Call the extract_vector for every frame of the Folder
currentframe = 0
feature_list = np.zeros(shape=(1, 2048))
while(currentframe<number_files):
    feature_list = extract_vector(cur_dir+path_to_cluster+"/"+list[currentframe],feature_list) # variavel
    currentframe = currentframe+1
    print(feature_list.shape)



In [ ]:
# Save the matrix of features in a .csv file to not run the cells of reduction above
name = cluster + "_list.csv"
savetxt(name, feature_list, delimiter=',')   #variavel 

In [ ]:
# Load the features in a .csv file to the feature_list
feature_list = loadtxt(name, delimiter=',') #variavel

In [ ]:
# Transform into a Dataframe
feature_list = pd.DataFrame(feature_list)
feature_list

In [ ]:
# Apply a reduction of dimension PCA algorithm
pca = PCA(0.90)
features_pca = pca.fit_transform(feature_list
)

# explained percentage of importance
pca.explained_variance_ratio_

In [ ]:
# Number of components of PCA 
pca.n_components_

In [ ]:
# Transform into a Dataframe
features_pca = pd.DataFrame(features_pca)
features_pca

In [ ]:
# Describe the stats of new matrix features
features_pca.describe()

In [ ]:
# Normalize the matrix of features 
scaler = MinMaxScaler()
scaler.fit(features_pca)
df_features_pca_scaled = scaler.transform(features_pca)
df_features_pca_scaled = pd.DataFrame(df_features_pca_scaled)
df_features_pca_scaled

In [ ]:
# Try or not to use the T-SNE algorithm to reduce the complexity of our features if - you doesnt use this cell use the next one.
tsne = TSNE(n_components=2)
features_tsne = tsne.fit_transform(df_features_pca_scaled)



In [ ]:
# Only get the matrix of features after the PCA and scaled.
features_tsne = df_features_pca_scaled

In [ ]:
# Normalize the matrix of features again
scaler = MinMaxScaler()
scaler.fit(features_tsne)
df_features_tsne_scaled = scaler.transform(features_tsne)
df_features_tsne_scaled = pd.DataFrame(df_features_tsne_scaled)
df_features_tsne_scaled

In [ ]:
def LeastSquareSolution(X,Y):       # multivariate regression
    transpose = X.T
    product = np.dot(transpose,X)
    inverse = np.linalg.pinv(product)
    product2 = np.dot(inverse,X.T)
    B = np.dot(Y,product2)
    return B    

In [ ]:
# Testing 1  removing Possible Outliers with a Random Sub-base
number_of_sub_space = 20
df_sub_base = df_features_tsne_scaled.sample(n=number_of_sub_space)

df_sub_base = np.array(df_sub_base,  dtype = float)
features = np.array(df_features_tsne_scaled,dtype = float)


Pi = np.dot(df_sub_base.T,df_sub_base)
lengh = int(len(Pi))
PiN = np.identity(lengh) - Pi
fi = np.dot(Pi,features.T)
fiN = np.dot(PiN,features.T)

di = np.sqrt(sum(np.multiply(fi, fi)))
dn = np.sqrt(sum(np.multiply(fiN, fiN)))
df = np.sqrt(sum(np.multiply(features, features)))

Ri = di / df [:,None]
Ri

Rin = dn / df [:,None]
Rin


In [ ]:
# Visualizing our Data

plt.scatter(df_features_tsne_scaled[0],df_features_tsne_scaled[1])
plt.show()
plt.hist(df_features_tsne_scaled[0], bins = 20, rwidth=0.8,density=True)
rng = np.arange(df_features_tsne_scaled[0].min(),df_features_tsne_scaled[0].max(),0.1)
plt.plot(rng, stats.norm.pdf(rng,df_features_tsne_scaled[0].mean(),df_features_tsne_scaled[0].std()))
plt.show()



In [ ]:
# Removing Possible Outilers Testing 2
list = pd.DataFrame(list)
df_features_tsne_scaled["frames"] = list[0]
df2 = df_features_tsne_scaled
for i in range(1,len(df2.columns-1)):
    min_thresold, max_thresold = df_features_tsne_scaled[i].quantile([0.001,0.999])
    df2 =df2 [(df_features_tsne_scaled[i]<min_thresold) & (df_features_tsne_scaled[i]>max_thresold)]
df2

df_frames = df2["frames"]
df_frames = pd.DataFrame(df_frames)
df2 = df2.drop("frames",axis="columns")

scaler = MinMaxScaler()
scaler.fit(df2)
df2 = scaler.transform(df2)
df_features_tsne_scaled = pd.DataFrame(df2)
df_features_tsne_scaled




In [ ]:
# The Elbow Method to know the K number of K-Means Custering
k_range = range(1,15)
sse = []

for k in k_range:
    km = KMeans(n_clusters=k)
    km.fit(df_features_tsne_scaled)
    sse.append(km.inertia_)

sse

In [ ]:
# Plot the Elbow Graph
plt.plot(k_range,sse)
plt.xlabel("K number")
plt.ylabel("Sum of Square Errors")
plt.title("K Value for Features")

In [ ]:
# Apply the K-Means CLustering
list = pd.DataFrame(list)
numero_clusters = 2
df_features_use = df_features_tsne_scaled
km = KMeans(n_clusters=numero_clusters)
cluster_features_predict = km.fit_predict(df_features_use)
df_features_use["cluster"] = cluster_features_predict
df_features_use["frames"] = list[0]
df_features_use

In [ ]:
# Testing the cluster 0 of features after K-Means Clustering
df = df_features_use[df_features_use.cluster == 0 ]
df

In [ ]:
# Plot of Cluster vs Frame to try understand our Cluster 
df_use = df_features_use

df1 = df_use[df_use.cluster==0]
df2 = df_use[df_use.cluster==1]
df3 = df_use[df_use.cluster==2]


plt.scatter(df1.frames,df1.cluster,color="red")
plt.scatter(df2.frames,df2.cluster,color="green")
plt.scatter(df3.frames,df3.cluster,color="blue")

plt.xlabel("frames")
plt.ylabel("cluster")
plt.legend()
plt.show()

# Plot of Cluster vs Frame to try understand our Cluster for the first 100 frames
plt.scatter(df1[0],df1[1],color="red")
plt.scatter(df2[0],df2[1],color="green")
plt.scatter(df3[0],df3[1],color="blue")

plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
# Create Folder for each Cluster with the all frames of them 

cur_dir = "/Users/natanviana/Desktop/big_data"
framesFolder = path_to_cluster +"/" # variavel

for x in range(numero_clusters):
    dirName2 = 'cluster%d' % x
    os.mkdir( cur_dir + framesFolder + dirName2)
    vector = df_features_use[df_features_use.cluster == x].frames
    vector = pd.DataFrame(vector)
    for y in vector.frames:
        print(y)
        img = Image.open(cur_dir + framesFolder +  y)
        img.save(cur_dir + framesFolder  + dirName2 +"/"+ y)

In [ ]:
# Create Folder for each Cluster with the all video Frames of them.

cur_dir = "/Users/natanviana/Desktop/big_data"
os.mkdir(cur_dir+ "/"+ nameVideo+"/clusterVideos/"+cluster)
for i in range(numero_clusters):
    os.chdir(cur_dir)
    dirName = "cluster%d" % i
    image_folder = '.'  # Use the folder
    video_name = dirName + '.avi'
    os.chdir(cur_dir + path_to_cluster +"/"+dirName)
    images = [img for img in os.listdir(image_folder) if img.endswith(".jpg") or
            img.endswith(".jpeg") or img.endswith("png")]

    fourcc = cv2.VideoWriter_fourcc(*'DIVX')

    # Array images should only consider
    # the image files ignoring others if any

    frame = cv2.imread(os.path.join(image_folder, images[0]))

    # setting the frame width, height width
    # the width, height of first image
    height, width, layers = frame.shape
    
    video = cv2.VideoWriter(cur_dir+ "/"+ nameVideo+"/clusterVideos/"+cluster + "/" + video_name, fourcc, 1, (width, height))

    # Appending the images to the video one by one
    count=0
    for image in images:
        count+=1
        
        video.write(cv2.imread(os.path.join(image_folder, image)))
    print(count)
    # Deallocating memories taken for window creation
    cv2.destroyAllWindows()
    video.release()  # releasing the video generated
    print("FINISHED")

In [ ]:
# View video that you choose by name of cluster , ex : cluster0.avi

cur_dir = "/Users/natanviana/Desktop/big_data"
os.chdir(cur_dir)
# Create a VideoCapture object and read from input file
import cv2
import time
videoName = input('What is the video you want to open?\n')
name = cur_dir + "/" +  nameVideo +"/clusterVideos/"+ cluster+ "/" + videoName
cap = cv2.VideoCapture(name)

# Check if camera opened successfully
if (cap.isOpened() == False):
    print("Error opening video  file")

# Read until video is completed
while (cap.isOpened()):
    time.sleep(0.5)
    # Capture frame-by-frame
    ret, frame = cap.read()
    print(ret)
    if ret == True:

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Break the loop
    else:
        break

# When everything done, release
# the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

In [ ]:

# Viewing 100 images per cluster

for i in range(numero_clusters):

    cluster_name = "cluster%d" % i
    fig = plt.figure(figsize=(70, 70), dpi=100,)
    gs = GridSpec(10, 10, wspace=0, hspace=0)
    listx = []
    for x in range(100):
        listx.append(fig.add_subplot(gs[x]))
    for ax in fig.get_axes():
        ax.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)

    
    path_of_the_directory = cur_dir + "/"+ nameVideo + "/clusterImages/"+cluster+ "/" + cluster_name  # onde vai buscar imagens
    number_images = 100  # define número de imagens a plotar
    rows = 10
    columns = 10

    # faz lista com todas as imagens
    filename = os.listdir(path_of_the_directory)
    for x in range(number_images):  # vai retirar 100 imagens
        chooseImg = random.choice(filename)  # chooses image at random
        filename.remove(chooseImg)  # removes this image from the list
        # reading images
        img = Image.open(path_of_the_directory + '/' + chooseImg)
        #  width, height = img.size
        listx[x].imshow(img, aspect='auto')
    plt.show()
